In [1]:
import numpy as np
import pandas as pd

In [19]:
supermarket_data = pd.read_csv("supermarket_sales - Sheet1.csv", index_col=0)

In [20]:
supermarket_data.head()

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
Invoice ID,,,,,,,,,,,,,,,,
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [21]:
supermarket_data['timeindex'] = pd.to_datetime(
    supermarket_data['Date'] + ' ' + supermarket_data['Time']
)

In [38]:
# choose features
supermarket_data = pd.read_csv(
    'supermarket_sales - Sheet1.csv',
    usecols=[
        'Invoice ID',
        'City',
        'Customer type',
        'Gender',
        'Product line',
        'Unit price',
        'Quantity',
        'Payment',
        'Rating',
        'Date',
        'Time'
    ]
)

# engineer gross sales feature
supermarket_data['sale_gross'] = round(
    supermarket_data['Unit price'] * supermarket_data['Quantity'],
    2
)

# rename columns
supermarket_data = supermarket_data.rename(
    columns={
        'Invoice ID': 'invoice',
        'City': 'branch',
        'Customer type': 'member',
        'Gender': 'gender',
        'Product line': 'category',
        'Unit price': 'unit_price',
        'Quantity': 'quantity',
        'Payment': 'payment_type',
        'Rating': 'rating',
        'Date': 'date',
        'Time': 'time'
    }
)

# set index
supermarket_data.set_index('invoice', inplace=True)

In [39]:
# verify changes
supermarket_data.head()

,branch,member,gender,category,unit_price,quantity,date,time,payment_type,rating,sale_gross
invoice,,,,,,,,,,,
750-67-8428,Yangon,Member,Female,Health and beauty,74.69,7,1/5/2019,13:08,Ewallet,9.1,522.83
226-31-3081,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3/8/2019,10:29,Cash,9.6,76.40
631-41-3108,Yangon,Normal,Male,Home and lifestyle,46.33,7,3/3/2019,13:23,Credit card,7.4,324.31
123-19-1176,Yangon,Member,Male,Health and beauty,58.22,8,1/27/2019,20:33,Ewallet,8.4,465.76
373-73-7910,Yangon,Normal,Male,Sports and travel,86.31,7,2/8/2019,10:37,Ewallet,5.3,604.17


In [40]:
# confirm changes did not affect total observations
supermarket_data.shape

(1000, 11)

In [41]:
# confirm datatypes for transfer to dashboard / report
supermarket_data.dtypes

branch           object
member           object
gender           object
category         object
unit_price      float64
quantity          int64
date             object
time             object
payment_type     object
rating          float64
sale_gross      float64
dtype: object

In [42]:
# convert to csv
supermarket_data.to_csv("supermarket_sales_modified.csv")